In [56]:
#CREATING DATA CREATION CLASS AND FUNCTIONS
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = True

class OCR():
    IMG_SIZE = 50
    LABELS = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'J': 19, 'K': 20, 'L': 21, 'M': 22, 'N': 23, 'O': 24, 'P': 25, 'Q': 26, 'R': 27, 'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'X': 33, 'Y': 34, 'Z': 35}
    training_data =[]
    testing_data = []
    count = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}
    total = 0
    def make_training_data(self):
        loc = "data2/training_data/"
        errors = 0
        for label in self.LABELS:
            
            for f in tqdm(os.listdir(os.path.join(loc, str(label)))):
                try:
                    path = os.path.join(loc, str(label), f)
                    img  = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(36)[self.LABELS[label]]])

                    self.count[label]+=1
                    self.total+=1
                except Exception as e:
                    errors+=1
                    print(e)
        self.training_data = np.array(self.training_data, dtype = 'object')
        np.random.shuffle(self.training_data)
        for i,j in self.count.items():
            print(i,":",j)
        print(self.total)
        print(errors)
        np.save("training_data.npy", self.training_data, allow_pickle = True)
        print(self.count)
        print(self.total)
        print(errors)
    
    def make_testing_data(self):
        loc = "data/testing_data/"
        errors = 0
        for label in self.LABELS:
            
            for f in tqdm(os.listdir(os.path.join(loc, str(label)))):
                try:
                    path = os.path.join(loc, str(label), f)
                    img  = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.testing_data.append([np.array(img), np.eye(36)[self.LABELS[label]]])

                except Exception as e:
                    errors+=1
                    
        self.testing_data = np.array(self.testing_data, dtype = 'object')
        np.random.shuffle(self.testing_data)
        for i,j in self.count.items():
            print(i,":",j)
        print(self.total)
        print(errors)
        np.save("testing_data.npy", self.testing_data, allow_pickle = True)
        


if REBUILD_DATA:
    ocr = OCR()
    ocr.make_training_data()
    ocr.make_testing_data()    

100%|██████████████████████████████████████████████████████████████████████████████| 573/573 [00:00<00:00, 1746.86it/s]


0 : 573
1 : 573
2 : 573
3 : 573
4 : 573
5 : 573
6 : 573
7 : 573
8 : 573
9 : 573
A : 573
B : 573
C : 573
D : 573
E : 573
F : 573
G : 573
H : 573
I : 573
J : 573
K : 573
L : 573
M : 573
N : 573
O : 573
P : 573
Q : 573
R : 573
S : 573
T : 573
U : 573
V : 573
W : 573
X : 573
Y : 573
Z : 573
20628
0
{0: 573, 1: 573, 2: 573, 3: 573, 4: 573, 5: 573, 6: 573, 7: 573, 8: 573, 9: 573, 'A': 573, 'B': 573, 'C': 573, 'D': 573, 'E': 573, 'F': 573, 'G': 573, 'H': 573, 'I': 573, 'J': 573, 'K': 573, 'L': 573, 'M': 573, 'N': 573, 'O': 573, 'P': 573, 'Q': 573, 'R': 573, 'S': 573, 'T': 573, 'U': 573, 'V': 573, 'W': 573, 'X': 573, 'Y': 573, 'Z': 573}
20628
0


100%|█████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 924.12it/s]

0 : 573
1 : 573
2 : 573
3 : 573
4 : 573
5 : 573
6 : 573
7 : 573
8 : 573
9 : 573
A : 573
B : 573
C : 573
D : 573
E : 573
F : 573
G : 573
H : 573
I : 573
J : 573
K : 573
L : 573
M : 573
N : 573
O : 573
P : 573
Q : 573
R : 573
S : 573
T : 573
U : 573
V : 573
W : 573
X : 573
Y : 573
Z : 573
20628
0


In [57]:
training_data = np.load("training_data.npy", allow_pickle = True)

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,7)
        self.conv2 = nn.Conv2d(32,64,7)
        self.conv3 = nn.Conv2d(64,128,7)
        self._to_linear = None
        #x = torch.flatten(self.conv3)
        #x = np.prod(x[0].shape)
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,36)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        #print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = np.prod(x[0].shape)
        
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)

    
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on GPU")
else:
    device = torch.device("cpu")
    print("running on CPU")
net = Net().to(device)

running on GPU


In [58]:
import torch.optim as optim


x = torch.Tensor([np.array(i[0]) for i in training_data]).view(-1,50,50)
x = x/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(x)*VAL_PCT)
print(val_size)

2062


In [59]:
train_x = x
train_y = y


In [64]:
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 10
    optimizer = optim.Adam(net.parameters(), lr = 0.001)
    loss_function = nn.MSELoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
            batch_x = train_x[i:i+BATCH_SIZE].view(-1,1,50,50)
            batch_y = train_y[i:i+BATCH_SIZE]
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            net.zero_grad()
            outputs = net(batch_x).to(device)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
    print(loss)

net = net.to(device)
train(net)

100%|████████████████████████████████████████████████████████████████████████████████| 207/207 [00:04<00:00, 51.15it/s]

tensor(3.5066e-05, device='cuda:0', grad_fn=<MseLossBackward0>)


In [5]:
#LOADING MODEL
#net.load_state_dict(torch.load('model_10ep.pt'))

<All keys matched successfully>

In [61]:
#MAKING TEST SET
testing_data = np.load("testing_data.npy", allow_pickle = True)

test_x = torch.Tensor([np.array(i[0]) for i in testing_data]).view(-1,50,50)
test_x = test_x/255.0

test_y = y = torch.Tensor([i[1] for i in testing_data])


In [65]:
#ACCURACY AGAINST TEST SET

correct  = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_x))):
        real_class = torch.argmax(test_y[i])
        net = net.to('cpu')
        net_out = net(test_x[i].view(-1,1,50,50))[0]
        pred_class = torch.argmax(net_out)
        if pred_class == real_class:
            correct+=1
        total += 1
print("Acc:",correct/total*100)

100%|█████████████████████████████████████████████████████████████████████████████| 1008/1008 [00:02<00:00, 338.33it/s]

Acc: 98.51190476190477


In [66]:
from datetime import datetime
t = str(datetime.now())
t = t[:10]
torch.save(net.state_dict(), f"model_{t}.pt")